# <center>DISCLAIMER !!!</center>                                    

1) Вся приведенная здесь информация даётся исключительно в просветительских целях. Пользуйтесь на свой страх и риск.    
2) Нет никакой гарантии, что описанный метод и соответствующий софт будет работать с будущими версиями защиты.  
3) Всё изложение ведётся примерно в том порядке, в котором я это исследовал. Если у Вас получится лучше, я за Вас рад, но кидаться в меня какашками право же не стоит.   
4) Ну и наконец, если приведённый здесь код **НЕ ЗАРАБОТАЕТ**, это ещё не значит что я плохой, и кидаться в меня вышеназванными предметами тоже не надо. Просто время жизни подобного кода - полгода-год, не более. Потом меняется дизайн сайта и в том числе защита. Меняется незначительно, но скраппер обычно привязывается к мелким деталям на страничке. Таким как имена классов и т.п. Однако обновить уже существующий скраппер гораздо легче, чем писать с нуля. 

In [ ]:
import requests as rqs
import time
from bs4 import BeautifulSoup as soup
import subprocess

Сервис **[hidemy.name](https://hidemy.name)**, использует защиту от **[Cloudflare](https://www.cloudflare.com/)**, которая якобы считается надёжной и позиционируется в том числе как защита от Ddos-атак. Как мы скоро увидим, это несколько не соответствует объективной реальности.  
Давайте для начала попробуем прочитать список прокси с **[hidemy.name](https://hidemy.name/en/proxy-list/)** самой низкоуровневой питоньей библиотекой **urllib**

In [ ]:
import urllib.request as urlrq
import urllib.error as err

hidemy_url="https://hidemy.name/en/proxy-list/"
try:
    urlrq.urlopen(hidemy_url)
except err.URLError as e :
    s=str(e.read().decode("utf-8"))
    file=open("hideme_403.html", "wt")
    file.write(s)
    file.close()
    print(e.code)

Сервер вернул код статуса 403 - **Forbidden**. Откроем сохранённый файл в браузере. Сразу видим, что **Access denied**, и пояснение, что хозяин сайта забанил нас из-за не понравившейся ему сигнатуры браузера. Что не удивительно. Настоящий браузер присылает в запросе множество дополнительной информации в заголовках, чего мы не сделали. Исправим это, попытаемся притвориться браузером:

In [ ]:
#Попробуйте здесь закомментировать или наоборот раскомментировать некоторые заголовки и посмотрите что получится    
rq=urlrq.Request(hidemy_url)
rq.add_header("User-Agent", "Mozilla/5.0 (X11; Linux x86_64; rv:59.0) Gecko/20100101 Firefox/59.0")
rq.add_header("Accept", "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8")
#rq.add_header("Accept-Encoding", "gzip, deflate")
#rq.add_header("Accept-Language", "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3")
#rq.add_header("Connection", "keep-alive")
try:
    urlrq.urlopen(rq)
except err.URLError as e :
    s=str(e.read().decode("utf-8"))
    file=open("hideme_503.html", "wt")
    file.write(s)
    file.close()   
    print(e.code)

Видим что код статуса теперь 503 - **Service Unavailable**. Открыв сохраненный файл в браузере видим, что нас просят подождать 5 секунд, пока наш якобы "браузер" проверяется. Этот пример показывает, насколько важны тут заголовки. И хотя в дальнейшем мы будем пользоваться более высокоуровневой библиотекой **requests**, которая по умолчанию вроде бы подставляет более вменяемые заголовки (в чем тут дело конкретно не разбирался), надёжнее будет собирать их ручками. Что и сделаем:

In [ ]:
# Заголовки из mozilla Firefox
firefox={
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",    
"Accept-Encoding": "gzip, deflate", 
"Accept-Language": "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3",
"Connection": "keep-alive",
"Host": "hidemy.name",
"Referer": "https://hidemy.name/en/",        
"Upgrade-Insecure-Requests": "1",    
"User-Agent":"Mozilla/5.0 (X11; Linux x86_64; rv:59.0) Gecko/20100101 Firefox/59.0"   
}

Посмотрим теперь на код сохранённого файла. Видно, что проверка сводится к выполнению некоего джаваскрипта. Посмотрев на скрипт, нетрудно понять, что он ждёт 4 секунды, после чего вычисляет немудряжий арифметический пример, записывает результат в скрытую форму и отправляет результат на сервер. Ничего сложного. Единственное что напрягает, это хитрый способ записи чисел. 


Например запись **(!+[]+!![]+!![]+!![]+!![]+!![]+!![]+!![]+!![]+[])** означает **9**.

На самом деле не бином Ньютона. Всё сводится к внутренним трюкам джаваскрипта при преобразовании типов в выражениях. Можно было бы достаточно просто написать интерпретатор для таких "чисел". Но во-первых это скучно. Во-вторых подобных трюков в джаваскрипте море. И мы не можем быть уверены, что нам не подкинут нечто, на чём интерпретатор сломается. Лучше выполнить пример на какой-то стандартной javascript-машине. 
 Для питона такая есть. Называется **PyV8** и представляет собой питонью оболочку движка **V8**. К сожалению у меня на дистрибутив Anaconda она почему-то отказалась ставиться. Нужна она мне не то чтоб очень, поэтому разбираться не стал. Ибо **node.js** стоит обычно у всех (у меня во всяком случае) и проще просто вызвать **node**.  
Итого, наши дальнейшие действия:  
1) Читаем страничку.  
2) Выделяем оттуда арифметический пример.  
3) Отдаём его javasript-машине.  
4) Забиваем ответ в форму.  
5) Отправляем форму на сервер.

In [ ]:
# Получает скрипт из странички
# Тут просто разные манипуляции с поиском и обрезанием строк, и немного джаваскрипта
def getScript(html):
    try:
        script="var "
        s=html[html.index("setTimeout"):]
        a=s.split("=")
        name=a[0][a[0].rfind(",")+1:].strip()
        script=script+name+"="
        a1=a[1].split('"')
        name=name+"."+a1[1]
        script=script+a[1]
        script=script[:script.index(";")+1]+"\n"
        script=script+html[html.index(name):]
        script=script[:script.index("value")]
        script=script[:script.rindex(";")+1]+"\n"
        script=script+'var t="hidemy.name";\n'
        script=script+"var a=("+name+"+t.length).toFixed(10);\n"
        script=script+"console.log(a);\n"
        return script
    except ValueError :
        return None    

Попробуем прочитать список прокси, выдернуть из странички скрипт и выполнить его в node

In [ ]:
nodename="node"                                  # если Вы под виндой, возможно это будет node.exe
rsp=rqs.get(hidemy_url, headers=firefox)         # читаем, передавая в запросе вменяемые заголовки
html=str(rsp.content.decode(rsp.encoding))       # получаем содержимое странички
file=open("hideme_start.html", "wt")             # сохраняем на всякий случай в файл  
file.write(html)
file.close()
print(rsp.headers)                               # смотрим заголовки ответа
script=getScript(html)                           # получаем скрипт и записываем в файл
file=open("script.js", "wt")                     # сохраняем на всякий случай в файл  
file.write(script)
file.close()
s=subprocess.Popen(nodename+" script.js", shell=True, stdout=subprocess.PIPE) # выполняем скрипт
s=s.stdout.read().decode("utf-8")               # получаем результат выполнения скрипта
print(s)

Видим, что сервер кроме проверочной странички прислал куку с именем **__cfduid**. Куки придется отрабатывать. Заведём под них переменную. И оформим все наши результаты в виде функции. Первая её версия будет только получать проверочную страничку, вытаскивать и выполнять скрипт из неё, и устанавливать куку.

In [ ]:
# Подключение к hidemy.name
cookies={}                                                                # для кук
def hidemyConnect1():
    global cookies
    cookies={}
    try:
        rsp=rqs.get(hidemy_url, headers=firefox, cookies=cookies)         # читаем страничку
        html=str(rsp.content.decode(rsp.encoding))    
        script=getScript(html)                                            # получаем и записываем скрипт
        if script==None:
            return None
        file=open("script.js", "wt")                       
        file.write(script)
        file.close()
        cc=rsp.cookies.get_dict()                                        # получаем и сохраняем куку
        cookies.update(cc)
        s=subprocess.Popen(nodename+" script.js", shell=True, stdout=subprocess.PIPE) # выполняем скрипт
        s=s.stdout.read().decode("utf-8")                                             # получаем результат 
        return s
    except:
        return None

In [ ]:
# Проверяем:
s=hidemyConnect1()
print(s)
print(cookies)

Теперь надо сформировать ответ якобы из формы и отправить его серверу  (отправка происходит методом GET). Параметры запроса (а этот запрос и есть наш ответ, прошу прощения за небольшую путаницу !) это пары имя-значение, взятые из формы. Нужно аккуратно распарсить форму, вытащить из неё имена и значения полей, сложить всё это в список параметров, а имени **jschl_answer** присвоить значение, выданное нашим скриптом. Удобнее всего это сделать с помощью питоньей библиотеки **BeautifulSoup**. Напишем новую версию нашей функции hidemyConnect.

In [ ]:
def hidemyConnect2():
    global cookies
    try:
        cookies={}
        rsp=rqs.get(hidemy_url, headers=firefox)      # читаем страничку
        html=str(rsp.content.decode(rsp.encoding))    
        script=getScript(html)                        # получаем и записываем скрипт
        if script==None:
            return None
        file=open("script.js", "wt")                       
        file.write(script)
        file.close()
        cc=rsp.cookies.get_dict()                    # получаем и сохраняем куку
        cookies.update(cc)
        s=subprocess.Popen(nodename+" script.js", shell=True, stdout=subprocess.PIPE) # выполняем скрипт
        s=s.stdout.read().decode("utf-8")                                             # получаем результат 
        # Обработка формы
        param={}                                     # словарь для параметров
        bsObj=soup(html, "lxml")                     # создаем объект BeautifulSoup
        form=bsObj.find("form")                      # находим форму
        inputs=form.find_all("input")                # находим в форме все теги input
        for i in range(0, len(inputs)):              # проходимся по всему найденному списку  
            name=inputs[i].attrs["name"]             # получаем атрибут name
            if name=="jschl_answer":                 # если имя jschl_answer, подсовываем туда результат скрипта
                value=s.strip()                      # обрезаем \n на конце
            else:
                value=inputs[i].attrs["value"]       # получаем атрибут value из формы
            param[name]=value                        # скидываем в список параметров
        action=form.attrs["action"]                  # получаем атрибут формы action
        url="https://hidemy.name"+action             # по этому адресу и надо посылать ответ
        return url, param
    except:
        return None
    
# Проверяем:
url, param=hidemyConnect2()
print(url)
print(param)

Теперь всё готово для того, чтобы отправить новый запрос серверу. Надо только подождать 4 секунды. Давайте сделаем это:

In [ ]:
url, param=hidemyConnect2()                                          # читаем страничку, получаем url и параметры
time.sleep(4)                                                        # ждем 4 секунды
rsp=rqs.get(url, headers=firefox, cookies=cookies, params=param)     # отправляем запрос
html=str(rsp.content.decode(rsp.encoding))
file=open("hideme_proxy.html", "wt")                                 # записываем на диск
file.write(html)
file.close()
print(rsp.status_code)                                               # печатаем статус

Уррааа !!! Получили статус 200 !!! Страничка нормально прочитана !!!  
Попробуем ещё раз !

In [ ]:
rsp1=rqs.get(hidemy_url, headers=firefox, cookies=cookies)         
html=str(rsp1.content.decode(rsp1.encoding))  
file=open("test.html", "wt")                                 
file.write(html)
file.close()
print(rsp1.status_code)  

Упссс... Здрассье я ваша тётя... Снова статус 503 и снова пришла проверочная страничка... Как же так ! Мы нормально прошли проверку и получили куку, а теперь начинать по новой ? Почему же в браузере всё работает ???   
Лезем в браузер смотреть куки. Видим, что среди кук, полученных от сайта **[hidemy.name](https://hidemy.name)**, две выделяются названиями, содержащими **cf**, что как бы намекает на **Cloudflare**.Одну из этих кук - **__cfduid**, мы уже получили, а другую **cf_clearance** - нет. Может она пришла в браузер  раньше ? Нет, не похоже. Но на всякий случай давайте проверим.   
Удаляем обе куки в браузере и снова заходим на **[hidemy.name](https://hidemy.name/en/proxy-list/)**. Снова проверка и через пять секунд список прокси. Снова смотрим куки. Чудеса... Обе на месте ! Наверно питонья библиотека **requests** глючит и выдаёт одну куку вместо положенных двух. Начинаем лопатить исходники...   
Наконец после пары-тройки мегабайт просмотренного кода и пары-тройки литров выпитого кофе(хорошо ещё что не после пары-тройки бессонных ночей !), вспоминаем, что существует такая хитрая штука как редирект запросов. Сервер как бы говорит клиенту мол погоди. Раньше тут был ресурс, но теперь он переехал. Сейчас я тебя перенаправлю. И перенаправляет. А за одно может прислать какую-нибудь дополнительную информацию. Например куку...    
В присылаемом питоньей библиотекой ответе это должно отражаться. Для чего в нем существует история - **history**. Смотрим историю:

In [ ]:
rsp2=rsp.history[0]
print(rsp2.status_code)
print(str(rsp2.content.decode(rsp2.encoding)))
print(rsp2.headers)

В истории лежит ответ с кодом 302 - **Moved Temporarily**, немудрящий html, и урааа !!! В заголовке нашлась та самая потерявшаяся кука **cf_clearance** !!! Напишем следующую версию нашей функции **hidemyConnect**, сохраняющую эту куку. К счастью новая версия и будет окончательной.   

Дополнительная мораль сей басни такова: **ВСЕ БАГИ У ВАС В КОДЕ !!!** Да, у этого правила тоже есть исключения. Я сам отлавливал баги и в библиотеках и в компиляторах, а однажды даже в системе команд процессора (не интел, не арм и не спарк). Однако если у Вас что-то упорно не получается, хорошо подумайте, прежде чем лезть в исходники библиотек. **ОЧЕНЬ** хорошо подумайте.

In [ ]:
def hidemyConnect():
    global cookies
    try:
        cookies={}
        rsp=rqs.get(hidemy_url, headers=firefox)      # читаем страничку
        html=str(rsp.content.decode(rsp.encoding))    
        script=getScript(html)                        # получаем и записываем скрипт
        if script==None:
            return None
        file=open("script.js", "wt")                       
        file.write(script)
        file.close()
        cc=rsp.cookies.get_dict()                    # получаем и сохраняем куку
        cookies.update(cc)
        s=subprocess.Popen(nodename+" script.js", shell=True, stdout=subprocess.PIPE) # выполняем скрипт
        s=s.stdout.read().decode("utf-8")                                             # получаем результат 
        # Обработка формы
        param={}                                     # словарь для параметров
        bsObj=soup(html, "lxml")                     # создаем объект BeautifulSoup
        form=bsObj.find("form")                      # находим форму
        inputs=form.find_all("input")                # находим в форме все теги input
        for i in range(0, len(inputs)):              # проходимся по всему найденному списку  
            name=inputs[i].attrs["name"]             # получаем атрибут name
            if name=="jschl_answer":                 # если имя jschl_answer, подсовываем туда результат скрипта
                value=s.strip()                      # обрезаем \n на конце
            else:
                value=inputs[i].attrs["value"]       # получаем атрибут value из формы
            param[name]=value                        # скидываем в список параметров
        action=form.attrs["action"]                  # получаем атрибут формы action
        url="https://hidemy.name"+action             # по этому адресу и надо посылать ответ
        # Отправляем проверочный ответ
        time.sleep(4)                                # ждем 4 секунды
        rsp=rqs.get(url, headers=firefox, cookies=cookies, params=param)     # отправляем запрос
        html=str(rsp.content.decode(rsp.encoding))
        rsp=rsp.history[0]
        cc=rsp.cookies.get_dict()                    # получаем и сохраняем куку
        cookies.update(cc)
        return html
    except:
        return None
    
    
# Проверяем:
hidemyConnect()
print(cookies)

Прекрасно ! В списке видны две куки. Снова пытаемся прочитать штатно:

In [ ]:
rsp1=rqs.get(hidemy_url, headers=firefox, cookies=cookies)         
html=str(rsp1.content.decode(rsp1.encoding))  
file=open("test.html", "wt")                                 
file.write(html)
file.close()
print(rsp1.status_code)  

Получили 200. Открывая сохраненный файл в браузере, видим список прокси. Грозная защита  **[Cloudflare](https://www.cloudflare.com/)** успешно преодолена ! Теперь мы можем заDdosить весь интернет ! Или что более миролюбиво и более практично, качать на халяву список прокси с **[hidemy.name](https://hidemy.name)**. Остался только последний шаг. Распарсить полученную страничку со списком прокси и вывести прокси в виде списка, чтобы этим можно было пользоваться. Это легко сделать с помощью той же библиотеки **BeautifulSoup**, примерно вот так:

In [ ]:
proxies=[]
bsObj=soup(html, "html.parser")                     
table=bsObj.find("tbody")                      
trs=table.find_all("tr")
tds=trs[0].find_all("td")
ip_addr=tds[0].contents[0]
port=tds[1].contents[0]
proxies.append([ip_addr, port])

In [ ]:
# Парсит страничку из hidemy.name
def parseHidemy(html):
    proxies=[]
    bsObj=soup(html, "html.parser")                     
    table=bsObj.find("tbody") 
    trs=table.find_all("tr")
    for i in range(0, len(trs)):
        tds=tds=trs[i].find_all("td")
        ip_addr=tds[0].contents[0]
        port=tds[1].contents[0]
        proxies.append([ip_addr, port])
    return proxies;

# Проверяем
proxies=parseHidemy(html)
print(proxies)

Остался только небольшой завершающий штрих картины. Ресурс **hidemy.name** довольно гибко управляет поиском прокси, позволяя выбрать страну расположения прокси, номер порта, скорость, тип протокола и уровень анонимности. Это может оказаться полезным. Сделаем его настройку из скрипта.   
На страничке для поиска прокси с заданными параметрами, служит кнопка **Show**. По коду странички ясно, что она связана с джаваскриптом **search()**, который осуществляет считывание настроек поиска, формирует из них параметры и делает GET-запрос. Так же параметром запроса передается номер странички, с которой читаются прокси. В каждой страничке по 64 прокси, и откуда читать, задаётся параметром **start** - первым номером прокси, с которого надо читать. Например для второй странички он будет 64.   

Разберёмся со странами. Как видно из кода странички, чекбоксы выбора стран заключены в теги **label**. Получить название и код страны можно примерно так: 

In [ ]:
bsObj=soup(html, "html.parser")                     
labels=bsObj.find_all("label")                      
label=labels[10]                      # к примеру
inp=label.find("input")
cls=inp.attrs["class"]
name=""
if "choose-country" in cls :
    name=label.contents[4].contents[0]
    name=name[:name.index("(")].strip()
    code=inp.attrs["id"]
    code=code[code.rindex("_")+1:]
    print(name)
    print(code)

In [ ]:
# Получение списка стран
def getCountries(bsObj):
    countries=[]
    labels=bsObj.find_all("label")                      
    for i in range(0, len(labels)):
        label=labels[i]
        inp=label.find("input")
        try:
            cls=inp.attrs["class"]
            if "choose-country" in cls :
                name=label.contents[4].contents[0]
                name=name[:name.index("(")].strip()
                code=inp.attrs["id"]
                code=code[code.rindex("_")+1:]
                countries.append([name, code, "yes"])    # в элемент добавлеям yes, чтобы страна вошла в поиск
            else : 
                print(label) # поскольку большинство тегов label содержат страны, выведем остальные чтобы
                         # разобраться с ними 
        except:
            print(label)
    return countries
# Проверяем
countries=getCountries(bsObj)
print(countries)

Видим, что распечатался тег, соответствующий выбору всех стран, теги для ввода скорости и портов, теги для выбора протоколов и теги для выбора уровня анонимности. Тег для выбора всех стран нам не нужен. Теги скорости и портов это текстовые поля ввода. Чекбоксы, содержащие названия и коды есть только в тегах протоколов и анонимности. Вытащим в виде списков и их тоже. Напишем новую функцию, вытаскивающую сразу и страны и протоколы, и анонимность:

In [ ]:
# Вытаскивает списки всех контролов
def parseControls(bsObj):
    countries=[]
    protocols=[]
    annons=[]
    labels=bsObj.find_all("label")                      
    for i in range(0, len(labels)):
        label=labels[i]
        inp=label.find("input")
        if "class" in inp.attrs.keys():
            cls=inp.attrs["class"]
            if "choose-country" in cls :
                name=label.contents[4].contents[0]
                name=name[:name.index("(")].strip()
                code=inp.attrs["id"]
                code=code[code.rindex("_")+1:]
                countries.append([name, code, "yes"])    
        elif "id" in inp.attrs.keys():
            if inp.attrs["id"][0]=="t":
                name=label.contents[2].contents[0]
                code=inp.attrs["id"][2:]
                protocols.append([name, code, "yes"])
            elif inp.attrs["id"][0]=="a":
                name=label.contents[2].contents[0]
                code=inp.attrs["id"][2:]
                annons.append([name, code, "yes"])    
    return countries, protocols, annons
    
# Проверяем
countries, protocols, annons=parseControls(bsObj)    
print(countries) 
print(protocols) 
print(annons)

На этом исследовательская часть работы закончена. Настала пора собрать всё до кучи. У нас тут довольно много вспомогательных переменных. Куки, списки, заголовки... Поэтому удобно будет оформить окончательный код в виде класса, чтобы оно без дела под ногами не путалось. Чтобы не загромождать зря блокнот и чтобы было удобнее писать, я вынес класс в отдельный питоний файл, а код писал в **[WingIDE](https://wingware.com/)**. Здесь в блокноте будем только проверять.

In [ ]:
from hidemyname import HidemyName
print("Готово !")

In [ ]:
hide=HidemyName()

In [ ]:
f=hide.connect()
f

In [ ]:
c=hide.getCoutriesList()
p=hide.getProtocolsList()
a=hide.getAnnonsList()

In [ ]:
#hide.getCountries()


In [ ]:
#hide.delCountry("Afghanistan")
#c

In [ ]:
#hide.addCountry("Afghanistan")

In [ ]:
#hide.addCountry(['Armenia', 'Australia', 'Austria'])
#c

In [ ]:
s="123"
type(s)
if type(s) != list:
    s=[s]
s

In [ ]:
import urllib.request as urlrq
import urllib.error as err

hidemy_url="https://www.proxyrotator.com/free-proxy-list/"
try:
    urlrq.urlopen(hidemy_url)
except err.URLError as e :
    s=str(e.read().decode("utf-8"))
    file=open("proxyrotator.html", "wt")
    file.write(s)
    file.close()
    print(e.code)

In [ ]:
#Попробуйте здесь закомментировать или наоборот раскомментировать некоторые заголовки и посмотрите что получится    
rq=urlrq.Request(hidemy_url)
rq.add_header("User-Agent", "Mozilla/5.0 (X11; Linux x86_64; rv:59.0) Gecko/20100101 Firefox/59.0")
rq.add_header("Accept", "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8")
#rq.add_header("Accept-Encoding", "gzip, deflate")
#rq.add_header("Accept-Language", "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3")
#rq.add_header("Connection", "keep-alive")
try:
    res=urlrq.urlopen(rq)
    s=str(res.read().decode("utf-8"))
    file=open("proxyrotator.html", "wt")
    file.write(s)
    file.close()   
    print(res.code)
except err.URLError as e :
    s=str(e.read().decode("utf-8"))
    file=open("proxyrotator.html", "wt")
    file.write(s)
    file.close()   
    print(e.code)

In [ ]:
res=urlrq.urlopen(rq)

In [ ]:
res

In [2]:
a=[1,2,3,4]
a

[1, 2, 3, 4]

На этом пожалуй всё. Если хотите, можете прочитать блокнот про **[proxyrotator](./proxyrotator_new.ipynb)**

In [5]:
a.append(5,6)

TypeError: append() takes exactly one argument (2 given)

In [6]:
a

[1, 2, 3, 4, 5]

In [7]:
a[0]

1

In [9]:
a[-2]

4

In [10]:
a

[1, 2, 3, 4, 5]

In [11]:
b=[6,7,8,9]

In [12]:
a+b

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [13]:
a=[0,1,2,"*",4,5,6,7,8,9]

In [14]:
a

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]